In [166]:
import networkx as nx
import matplotlib.pyplot as plt 
from pymystem3 import Mystem as mystem
import re
from nltk.tokenize import RegexpTokenizer as rt
from stop_words import get_stop_words
stop_words = get_stop_words('ru')
from nltk import FreqDist
import os
import itertools

## Создаем список текстов

In [197]:
list_of_texts = []
for some_file in os.listdir ('./Corpus'): 
    if some_file.endswith ('.txt'):
        with open (os.path.join('./Corpus', some_file),'r') as open_file:
            list_of_texts.append(open_file.read())

## Создаем функцию для лемматизации одного текста

In [67]:
def lemmatization_of_one_text(text):
    mystem = Mystem()
    lemmas = mystem.lemmatize(text)
    for word_idx in range(len(lemmas)):
        lemmas[word_idx] = lemmas[word_idx].strip('\n').strip('\xa0 ')
    lemmas = ' '.join(lemmas)
    clean_lemmas = re.sub('\d+?', '', lemmas)
    return clean_lemmas

## Создадим функцию, которая пролемматизирует нам все тексты

In [107]:
def lemmas_of_all_texts(list_of_texts):
    lemmas_of_all_texts = []
    for i in range(len(list_of_texts)):
        lemmas_of_all_texts.append(lemmatization(list_of_texts[i]))
    return lemmas_of_all_texts   

In [108]:
list_of_lemmas = lemmas_of_all_texts(list_of_texts)

## То же самое создадим для токенизации

In [109]:
def tokenization(text):
    tokenizer = rt('\w+')
    tokenized_lemmas = tokenizer.tokenize(text)
    return tokenized_lemmas

In [110]:
def tokenization_of_all_texts(list_of_lemmas):
    tokens_of_all_texts = []
    for i in range(len(list_of_texts)):
        tokens_of_all_texts.append(tokenization(list_of_lemmas[i]))
    return tokens_of_all_texts   

In [142]:
list_of_tokenized_texts = tokenization_of_all_texts(list_of_lemmas)

## Теперь почистим от стоп-слов (в какой-то степени это опционально, особенно, если мы работаем с биграмами)

In [97]:
def drop_stop_words_from_text_list(list_of_tokenized_texts):
    for i in range(len(list_of_tokenized_texts)):
        for stop_word in stop_words:
            list_of_tokenized_texts[i] = list(filter(lambda word: word != stop_word, list_of_tokenized_texts[i]))
    return list_of_tokenized_texts

In [144]:
no_stop_words = drop_stop_words_from_text_list(list_of_tokenized_texts)

In [145]:
zoshenko = no_stop_words[0]

In [175]:
QGIS = no_stop_words[1]

In [176]:
N_1 = no_stop_words[2]

In [148]:
Gus = no_stop_words[3]

## Попробуем создать пары слов для каждого нашего текста

In [124]:
def couple(list_of_tokens):
    couples = []
    for i, word in enumerate(list_of_tokens[:-1]):
        couple = word + ' '+ list_of_tokens[i+1]
        couples.append(couple)
    return couples

In [137]:
zoshenko_couples = couple(list_of_tokenized_texts[0])

In [177]:
QGIS_couples  = couple(list_of_tokenized_texts[1])

In [178]:
N_1_couples  = couple(list_of_tokenized_texts[2])

In [140]:
Gus_couples  = couple(list_of_tokenized_texts[3])

## Теперь создадим словари частотности. Первый подходит просто для подсчета слов, в то время как второй нужен для биграмов, причем в нем дуплеты типа "он пошел" и "пошел он" слиты воедино

In [135]:
def counting_words(list_of_words):
    dict_with_quantity = {}
    for word in list_of_words:
        if word in dict_with_quantity:
            dict_with_quantity[word] +=1
        else:
            dict_with_quantity[word]=1
    return dict_with_quantity

In [14]:
def sum_similar_pair_values(pair_dic):
    filter_pair_dic = {}
    for key1 in pair_dic.keys():
        for key2 in pair_dic.keys():
            if key1[0] == key2[1] and key1[1] == key2[0] and key1[0] < key1[1]:
                filter_pair_dic[key1] = pair_dic[key1] + pair_dic[key2]
    return filter_pair_dic

In [155]:
def sort(final_dictionary):
    itog = dict(sorted(final_dictionary.items(), key=lambda x: x[1], reverse=True))
    return itog

In [171]:
def dict_slice(dictionary, index):
    sliced_dictionary = dict(itertools.islice(sort(counting_words(dictionary)).items(), index))
    return sliced_dictionary

In [191]:
def dict_of_couples_slice(dictionary, index):
    sliced_dictionary = dict(itertools.islice(sort(sum_similar_pair_values(counting_words(dictionary))).items(), index))
    return sliced_dictionary

## Итак, время посмотреть на частотности!

### Частотности для одиноких слов

In [182]:
dict_slice(zoshenko, 10)

{'говорить': 24,
 'выходить': 9,
 'пьеса': 9,
 'гражданин': 7,
 'публика': 6,
 'купец': 6,
 'кричать': 6,
 'любитель': 5,
 'играть': 4,
 'дескать': 4}

In [183]:
dict_slice(N_1, 10)

{'экзопланета': 11,
 'проксим': 8,
 'звезда': 6,
 'обнаруживать': 6,
 'астроном': 5,
 'наблюдение': 5,
 'объект': 4,
 'кандидат': 4,
 'помощь': 4,
 'телескоп': 3}

In [184]:
dict_slice(QGIS, 10)

{'карта': 15,
 'данные': 10,
 'гис': 9,
 'проекция': 7,
 'система': 6,
 'работа': 5,
 'например': 5,
 'изображение': 5,
 'QGIS': 4,
 'использоваться': 4}

In [185]:
dict_slice(Gus, 10)

{'гусь': 64,
 'птица': 10,
 'конец': 7,
 'роспись': 7,
 'гусыня': 7,
 'город': 7,
 'римлянин': 6,
 'лебедь': 6,
 'сократ': 6,
 'матушка': 6}

### Частотности для пар слов

In [192]:
dict_of_couples_slice(zoshenko_couples, 10)

{'видеть не': 5,
 'виноватый из': 4,
 'видеть один': 4,
 'видеть крик': 4,
 'ерунда ничто': 3,
 'если подумать': 3,
 'до гора': 3,
 'публика дура': 3,
 'истинный происшествие': 2,
 'астрахань рассказывать': 2}

In [193]:
dict_of_couples_slice(N_1_couples, 10)

{'астроном обнаруживать': 3,
 'астроном подводить': 2,
 'комплексный наблюдение': 2,
 'опубликовывать сайт': 2,
 'существование звезда': 2,
 'суперземля минимальный': 2,
 'орбитальный период': 2,
 'большой полуось': 2,
 'орбита астрономический': 2,
 'астрономический единица': 2}

In [194]:
dict_of_couples_slice(QGIS_couples, 10)

{'анализ статистический': 3,
 'горный высота': 2,
 'большой данные': 2,
 'использоваться случай': 2,
 'оставаться связанный': 2,
 'исследовать мнение': 2,
 'официальный статистический': 2,
 'использоваться разный': 2,
 'отрасль литература': 2,
 'источник черпать': 2}

In [196]:
dict_of_couples_slice(Gus_couples, 10)

{'дикий гусь': 4,
 'зимний солнцестояние': 3,
 'конец лето': 3,
 'до наш': 3,
 'домашний гусь': 3,
 'история рим': 3,
 'конец конец': 3,
 'который так': 3,
 'город гусь': 3,
 'осенний равноденствие': 2}